In [8]:
import pandas as pd
import os
# C:\Users\t-yiranwu\OneDrive - Microsoft\Desktop\SecRL\secgym\env\data\alpineSkiHouse\SecurityAlert
# secgym\env\data\alpineSkiHouse\SecurityAlert

incident_path = "./data_anonymized/alphineskihouse/SecurityIncident.csv"
sep = "❖"

security_incident = pd.read_csv(incident_path, sep=sep, encoding='utf-8', on_bad_lines='skip', engine='python')
# combine all the alert files

alert_path_1 = './data_anonymized/alphineskihouse/SecurityAlert/SecurityAlert_0.csv'
alert_path_2 = './data_anonymized/alphineskihouse/SecurityAlert/SecurityAlert_1.csv'
security_alert =  pd.read_csv(alert_path_1, sep=sep, encoding='utf-8', on_bad_lines='skip', engine='python')
security_alert_2 = pd.read_csv(alert_path_2, sep=sep, encoding='utf-8', on_bad_lines='skip', engine='python')

# combine them in pd
security_alert = pd.concat([security_alert, security_alert_2])

In [9]:
# sort by time
security_incident["TimeGenerated"] = pd.to_datetime(security_incident["TimeGenerated"], format='mixed')
security_incident = security_incident.sort_values(by="TimeGenerated")

security_alert["TimeGenerated"] = pd.to_datetime(security_alert["TimeGenerated"], format='mixed')
security_alert = security_alert.sort_values(by="TimeGenerated")

In [10]:
# get earliest and latest time for security incident
earliest_time = security_incident["TimeGenerated"].min()
latest_time = security_incident["TimeGenerated"].max()

earliest_time, latest_time

(Timestamp('2024-07-02 22:41:59.791774+0000', tz='UTC'),
 Timestamp('2024-08-02 22:41:44.537137+0000', tz='UTC'))

In [11]:
import json

# 1. Filter by LATEST label
filtered_incidents = []
for i, incident in security_incident.iterrows():
    labels = json.loads(incident['Labels'])
    # check if the incident is the latest in one line
    is_latest = any([label['labelName'] == 'LATEST' for label in labels])
    if is_latest:
        filtered_incidents.append(incident)

filtered_incidents = pd.DataFrame(filtered_incidents)

# unique incident numbers
incident_numbers = filtered_incidents['IncidentNumber'].unique()
incident_numbers

array([ 55,  38,   5,   3,  39,  34, 122, 134, 166, 175, 290, 322])

In [12]:
# 2. for each incdent number, get the latest incident generated and put in a new dataframe
latest_incidents = []
for incident_number in incident_numbers:
    incident = filtered_incidents[filtered_incidents['IncidentNumber'] == incident_number]
    latest_incidents.append(incident.iloc[-1])
    
latest_incidents = pd.DataFrame(latest_incidents)
len(latest_incidents)

12

In [13]:
# 3. filter by time and remove repeated incidents
start_time = pd.to_datetime("2024-06-18T00:00:00Z")
end_time = pd.to_datetime("2024-08-02T00:00:00Z")

latest_incidents["FirstActivityTime"] = pd.to_datetime(latest_incidents["FirstActivityTime"], format='mixed')
latest_incidents = latest_incidents[latest_incidents["FirstActivityTime"] > start_time]
latest_incidents = latest_incidents[latest_incidents["FirstActivityTime"] < end_time]


# remove 290, 175, 122
latest_incidents = latest_incidents[latest_incidents['IncidentNumber'] != 290]
latest_incidents = latest_incidents[latest_incidents['IncidentNumber'] != 175]
latest_incidents = latest_incidents[latest_incidents['IncidentNumber'] != 122]

print(len(latest_incidents))

incident_numbers = latest_incidents['IncidentNumber'].unique()
incident_numbers

8


array([ 55,  38,   5,  39,  34, 134, 166, 322])

In [14]:
# print alert names
def get_alert_titles(alert_ids: list):
    alert_titles = []
    for alert_id in alert_ids:
        alert = security_alert[security_alert['SystemAlertId'] == alert_id]
        if len(alert) > 0:
            alert_titles.append(alert.iloc[0]['AlertName'])
    return alert_titles

def check_alert_name_overlap(incident_1, incident_2):
    alert_ids_1 = json.loads(incident_1['AlertIds'])
    alert_ids_2 = json.loads(incident_2['AlertIds'])
    alert_titles_1 = get_alert_titles(alert_ids_1)
    alert_titles_2 = get_alert_titles(alert_ids_2)
    overlap = set(alert_titles_1).intersection(set(alert_titles_2))
    return len(alert_ids_1), len(alert_ids_2), len(overlap)

def compare_incidents(incident_1, incident_2):
    print(incident_1['Title'])
    print(incident_2['Title'])
    labels_1 = [label['labelName'] for label in json.loads(incident_1['Labels'])]
    labels_2 = [label['labelName'] for label in json.loads(incident_2['Labels'])]
    print(labels_1)
    print(labels_2)
    
    len_alerts_1, len_alerts_2, overlap = check_alert_name_overlap(incident_1, incident_2)
    print(len_alerts_1, len_alerts_2, len(overlap))
    
    
# get overlap of every two incidents
for i, incident_1 in latest_incidents.iterrows():
    for j, incident_2 in latest_incidents.iterrows():
        if i < j:
            _, _, overlap = check_alert_name_overlap(incident_1, incident_2)
            if overlap ==0:
                continue
            print(incident_1['IncidentNumber'], incident_2['IncidentNumber'], check_alert_name_overlap(incident_1, incident_2))

55 34 (28, 11, 2)
5 55 (71, 28, 2)
5 39 (71, 48, 7)
39 55 (48, 28, 6)
134 55 (9, 28, 2)
134 39 (9, 48, 2)
134 166 (9, 11, 5)
134 322 (9, 9, 2)
322 55 (9, 28, 3)
322 39 (9, 48, 2)


In [15]:
latest_incidents.iloc[0]

TenantId                               e34d562e-ef12-4c4e-9bc0-7c6ae357c015
TimeGenerated                              2024-07-22 22:36:59.791921+00:00
IncidentName                           c5085ef5-1d1d-441e-b1d7-d8ae258445e8
Title                     Multi-stage incident involving Initial access ...
Description                                                             NaN
Severity                                                               High
Status                                                               Active
Classification                                                          NaN
ClassificationComment                                                   NaN
ClassificationReason                                                    NaN
Owner                     {"objectId":null,"email":null,"assignedTo":"Mi...
ProviderName                                                  Microsoft XDR
ProviderIncidentId                                                      384
FirstActivit

In [16]:
# put the incidents to a dict:  IncidentNumber: firstActivityTime, lastActivityTime,
# convert from pd timestamp to  datetime object
    # 322: {
    #     "start_time": datetime(2024, 8, 1, 11, 0, 0, 0, tzinfo=timezone.utc),
    #     "end_time": datetime(2024, 8, 1, 14, 0, 0, 0, tzinfo=timezone.utc),
    # },

incidents = {}
for i, incident in latest_incidents.iterrows():
    incidents[incident["IncidentNumber"]] = {
        "start_time": pd.to_datetime(incident["FirstActivityTime"]).to_pydatetime(),
        "end_time": pd.to_datetime(incident["LastActivityTime"]).to_pydatetime(),
    }
    

In [17]:
incidents

{55: {'start_time': datetime.datetime(2024, 7, 1, 15, 1, 28, tzinfo=datetime.timezone.utc),
  'end_time': datetime.datetime(2024, 7, 7, 0, 1, 1, 21452, tzinfo=datetime.timezone.utc)},
 38: {'start_time': datetime.datetime(2024, 6, 26, 15, 49, 16, 784267, tzinfo=datetime.timezone.utc),
  'end_time': datetime.datetime(2024, 6, 26, 16, 13, 56, 115283, tzinfo=datetime.timezone.utc)},
 5: {'start_time': datetime.datetime(2024, 6, 20, 8, 51, 7, 52079, tzinfo=datetime.timezone.utc),
  'end_time': datetime.datetime(2024, 6, 20, 9, 38, 4, 116591, tzinfo=datetime.timezone.utc)},
 39: {'start_time': datetime.datetime(2024, 6, 27, 14, 25, 58, 353842, tzinfo=datetime.timezone.utc),
  'end_time': datetime.datetime(2024, 6, 27, 22, 21, 6, 820675, tzinfo=datetime.timezone.utc)},
 34: {'start_time': datetime.datetime(2024, 6, 26, 11, 57, 25, 302556, tzinfo=datetime.timezone.utc),
  'end_time': datetime.datetime(2024, 6, 26, 13, 17, 18, 874531, tzinfo=datetime.timezone.utc)},
 134: {'start_time': dateti

In [18]:
latest_incidents.value_counts("Title")

Title
BEC financial fraud attack was launched from a compromised account (attack disruption)                                 1
HuLuke-operated ransomware attack was launched from a compromised asset (attack disruption)                            1
Multi-stage incident involving Defense evasion & Discovery on one endpoint                                             1
Multi-stage incident involving Execution & Discovery on one endpoint                                                   1
Multi-stage incident involving Initial access & Collection on multiple endpoints reported by multiple sources          1
Multi-stage incident involving Initial access & Lateral movement on multiple endpoints reported by multiple sources    1
Multi-stage incident involving Privilege escalation on multiple endpoints reported by multiple sources                 1
SAP financial process Lukeipulation (attack disruption)                                                                1
Name: count, dtype: int64

In [19]:
# sort incidents by TimeGenerated
latest_incidents = latest_incidents.sort_values(by="FirstActivityTime")
# print name, first activity time, title
for i, incident in latest_incidents.iterrows():
    print(incident['FirstActivityTime'], incident['IncidentNumber'],  incident['Title'], len(json.loads(incident['AlertIds'])))

2024-06-20 08:51:07.052079+00:00 5 HuLuke-operated ransomware attack was launched from a compromised asset (attack disruption) 71
2024-06-26 11:57:25.302556+00:00 34 Multi-stage incident involving Execution & Discovery on one endpoint 11
2024-06-26 15:49:16.784267+00:00 38 Multi-stage incident involving Defense evasion & Discovery on one endpoint 4
2024-06-27 14:25:58.353842+00:00 39 Multi-stage incident involving Privilege escalation on multiple endpoints reported by multiple sources 48
2024-07-01 15:01:28+00:00 55 Multi-stage incident involving Initial access & Lateral movement on multiple endpoints reported by multiple sources 28
2024-07-17 10:49:35.108080+00:00 134 BEC financial fraud attack was launched from a compromised account (attack disruption) 9
2024-07-22 08:18:18.418000+00:00 166 SAP financial process Lukeipulation (attack disruption) 11
2024-08-01 12:26:22+00:00 322 Multi-stage incident involving Initial access & Collection on multiple endpoints reported by multiple sourc

In [20]:
for i, incident in latest_incidents.iterrows():
    print(incident['IncidentNumber'])

5
34
38
39
55
134
166
322


## Use overlap in alert names to find relation beween two incidents

1. Have a large number of overlapping tags: like 19 and 25
2. Have the same incident title

In [21]:
# print alert names
def get_alert_titles(alert_ids: list):
    alert_titles = []
    for alert_id in alert_ids:
        alert = security_alert[security_alert['SystemAlertId'] == alert_id]
        if len(alert) > 0:
            alert_titles.append(alert.iloc[0]['AlertName'])
    return alert_titles

def check_alert_name_overlap(incident_1, incident_2):
    alert_ids_1 = json.loads(incident_1['AlertIds'])
    alert_ids_2 = json.loads(incident_2['AlertIds'])
    alert_titles_1 = get_alert_titles(alert_ids_1)
    alert_titles_2 = get_alert_titles(alert_ids_2)
    overlap = set(alert_titles_1).intersection(set(alert_titles_2))
    return len(alert_ids_1), len(alert_ids_2), len(overlap)

def compare_incidents(incident_1, incident_2):
    print(incident_1['Title'])
    print(incident_2['Title'])
    labels_1 = [label['labelName'] for label in json.loads(incident_1['Labels'])]
    labels_2 = [label['labelName'] for label in json.loads(incident_2['Labels'])]
    print(labels_1)
    print(labels_2)
    
    len_alerts_1, len_alerts_2, overlap = check_alert_name_overlap(incident_1, incident_2)
    print(len_alerts_1, len_alerts_2, len(overlap))
    
    
# get overlap of every two incidents
for i, incident_1 in latest_incidents.iterrows():
    for j, incident_2 in latest_incidents.iterrows():
        if i < j:
            _, _, overlap = check_alert_name_overlap(incident_1, incident_2)
            if overlap ==0:
                continue
            print(incident_1['IncidentNumber'], incident_2['IncidentNumber'], check_alert_name_overlap(incident_1, incident_2))

5 39 (71, 48, 7)
5 55 (71, 28, 2)
39 55 (48, 28, 6)
55 34 (28, 11, 2)
134 39 (9, 48, 2)
134 55 (9, 28, 2)
134 166 (9, 11, 5)
134 322 (9, 9, 2)
322 39 (9, 48, 2)
322 55 (9, 28, 3)
